In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

table_dictionary = pd.read_csv("data/TabDictionary.csv")
month_dictionary = pd.read_csv("data/TabMonth.csv")

dict = pd.Series(table_dictionary.TableNameFanGraphs.values,
                 index=table_dictionary.index).to_dict()

dict_month = pd.Series(month_dictionary.month.values,
                 index=month_dictionary.index).to_dict()

str_tab = 0
url = 'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=y&type={0}&season=2021&month=0&season1=2021&ind=0'.format(str_tab)
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')

for tab in range (0,25):
    str_tab = str(tab)
    table1 = soup.find('table', id='LeaderBoard1_dg1_ctl00')
    headers = []
    for i in table1.find_all('th'):
        title = i.text
        headers.append(title)        
    df = pd.DataFrame(columns = headers)
    # 1/7
    for year in range(2021,2023):
        minIP = 0
        
        url = 'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual={0}&type={1}&season={2}&month=0&season1={2}&ind=0'.format(minIP, str_tab, year)
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'lxml')
        div_pages = soup.find("div", {"class": "rgWrap rgInfoPart"})
        texts = [r.text.strip() for r in div_pages]
        pages = int(texts[3])
           
        table1 = soup.find('table', id='LeaderBoard1_dg1_ctl00')
        headers = []
        for i in table1.find_all('th'):
            title = i.text
            headers.append(title)  
        mydataComplete = pd.DataFrame(columns = headers)
    
        for page_number in range(0, pages):
            page_number_page = page_number+1
            url = 'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual={0}&type={1}&season={2}&month=0&season1={2}&ind=0&page={3}_30'.format(minIP, str_tab, year, page_number_page)
            
            # Create object page
            page = requests.get(url)
            soup = BeautifulSoup(page.text, 'lxml')
            table1 = soup.find('table', id='LeaderBoard1_dg1_ctl00')
            headers = []
            for i in table1.find_all('th'):
                title = i.text
                headers.append(title)            
            mydata = pd.DataFrame(columns = headers)
            
            idx = 0
            for j in table1.find_all('tr')[1:]:
                row_data = j.find_all('td')
                row = [i.text for i in row_data]
                length = len(mydata)
                if idx<=1:
                    idx=idx+1
                    continue
                mydata.loc[length] = row
            mydata['Year'] = year
            mydata['YearDate'] = ('{0}-01-01').format(str(year))
            mydata['YearDate']= pd.to_datetime(mydata['YearDate'])
            frames = [mydataComplete, mydata]
            df_pages = pd.concat(frames)
            mydataComplete = df_pages           
        frames = [df, df_pages]
        df_year = pd.concat(frames)
        df = df_year     
    df_year=df_year.dropna(axis=1,how='all')
    aa = dict.get(tab)
    df_year.to_csv('data/{0}/Picherts{0}FullSeason{1}.csv'.format(dict.get(tab), year), index=False)
    del df
    del df_year             